In [ ]:
library(Seurat)
library(openxlsx)
library(readxl)
library(Matrix)
library(readr)


In [3]:
#Need to find out if all duplicates are nonzeros
#Tostipanc
setwd("/dh-projects/ag-conrad/analysis/saal11/panfib/Renaming/ESPACE")
#Upload matrices
data_espace_matrix <- readMM("data_espace_matrix.mtx")
#Upload original gene list
data_espace_genes <- read.delim("data_espace_genes.tsv")
data_espace_genes <- data_espace_genes$genes
data_espace_cellIDs <- read.delim("data_espace_cellIDs.tsv")
rownames(data_espace_matrix) <- data_espace_genes
colnames(data_espace_matrix) <- data_espace_cellIDs$cellIDs

In [23]:
#Upload list of duplicates
data_espace_genes_dup <- read.xlsx("data_espace_geneIDs_duplicates_forsum.xlsx")

In [24]:
library(future)
plan("multicore", workers = 6)

In [26]:
# Initialize an empty list to store the sums
sums_list <- list()

# Iterate over each gene in your list
for (gene in data_espace_genes_dup$Old) {
  # Subset the count matrix for the current gene and calculate the sum
  gene_sum <- sum(data_espace_matrix[rownames(data_espace_matrix) == gene, ])
  
  # Add the sum to the sums_list with the gene name as the key
  sums_list[[gene]] <- gene_sum
}

In [ ]:
sums_list

In [28]:
#Upload manually editted list
data_espace_genes <- read.xlsx("data_espace_geneIDs_int_removed_manualedit.xlsx")

In [29]:
#Plug new genes to the matrix
rownames(data_espace_matrix) <- data_espace_genes$New

In [31]:
#Remove zeros
row_sums <- rowSums(data_espace_matrix)
names_with_zeros <- names(row_sums[row_sums == 0])
length(names_with_zeros)

[1] 4286

In [32]:
filtered_data_espace_matrix <- data_espace_matrix[row_sums != 0, ]

In [33]:
#Check for duplicates. Do they match the genes to be concatenated?
duplicate_genes <- duplicated(rownames(filtered_data_espace_matrix))
cat("Duplicate gene IDs:\n")
print(rownames(filtered_data_espace_matrix)[duplicate_genes])
duplicated <- rownames(filtered_data_espace_matrix)[duplicate_genes]


Duplicate gene IDs:
 [1] "GPR89B"     "LYPLAL1-DT" "LINC02814"  "LINC01819"  "LINC01291" 
 [6] "MCM2"       "SLC4A4"     "CCNA2"      "ROPN1L"     "LY6G6F"    
[11] "KHDRBS2"    "GJA9"       "UBE2R2-AS1" "KRT6A"      "TBC1D3B"   
[16] "SOCS4"      "TMC6"       "IGFLR1"     "PSG9"       "KLKB1"     
[21] "SIGLEC12"   "MMRN2"     


In [34]:
#Concatenating replicates
filtered_data_espace_matrix <- as(filtered_data_espace_matrix, "dgCMatrix")

'as(<dgTMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "CsparseMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").



In [35]:
gene_names <- rownames(filtered_data_espace_matrix)
unique_gene_names <- unique(gene_names)
gene_factors <- factor(gene_names, levels = unique_gene_names)

In [36]:
aggregated_counts <- rowsum(as.matrix(filtered_data_espace_matrix), group = gene_factors)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 46.2 GiB”


In [38]:
str(filtered_data_espace_matrix)
str(aggregated_counts)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:329759993] 9 77 90 119 128 130 171 183 187 195 ...
  ..@ p       : int [1:191695] 0 2958 5056 7444 9958 12171 14567 16920 19188 21394 ...
  ..@ Dim     : int [1:2] 32315 191694
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:32315] "MIR1302-2HG" "AL627309.1" "AL627309.2" "AL627309.5" ...
  .. ..$ : chr [1:191694] "APSE-control-pancreas-01-a_ATACCTTTCGTGCACG-1" "APSE-control-pancreas-01-a_CTCCCTCTCGCCCAGA-1" "APSE-control-pancreas-01-a_CAATACGTCCACTGGG-1" "APSE-control-pancreas-01-a_AGGGCCTTCTTTGCTA-1" ...
  ..@ x       : num [1:329759993] 1 3 1 2 1 2 1 1 1 5 ...
  ..@ factors : list()
 num [1:32293, 1:191694] 0 0 0 0 0 0 0 0 0 1 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:32293] "MIR1302-2HG" "AL627309.1" "AL627309.2" "AL627309.5" ...
  ..$ : chr [1:191694] "APSE-control-pancreas-01-a_ATACCTTTCGTGCACG-1" "APSE-control-pancreas-01-a_CTCCCTCTCGCCCAGA-1" "APSE-control-pancreas-01-a_CAATACGTCCACTGGG-1" "APSE

In [39]:
#Creating a seurat object
#Load metadata and dim reductions
data_espace_metadata <- read_csv("data_espace_metadata_final.csv")

New names:
• `` -> `...1`
• `...1` -> `...2`
Rows: 191694 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (15): ...2, sex, condition, subclass, aetiology, location, study, test, ...
dbl  (5): ...1, nCount_RNA, nFeature_RNA, age, percent.mt

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [40]:
data_espace_metadata <- as.data.frame(data_espace_metadata)

In [42]:
rownames(data_espace_metadata) <- data_espace_metadata$`...2`

In [43]:
data_espace_metadata$`...2` <- NULL
data_espace_metadata$`...1` <- NULL

In [46]:
data_espace_metadata$test <- NULL

In [47]:
data_espace_metadata

,nCount_RNA,nFeature_RNA,age,sex,condition,subclass,aetiology,percent.mt,location,study,cells,lineage,patientID,fileID,tissue,primary_or_cultured,cells_or_nuclei
,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
APSE-control-pancreas-01-a_ATACCTTTCGTGCACG-1,6826,2958,53,F,fibrotic-panc,Endothel_venous,fibrosis,0.003808966,head_PPI,Liebig,Endothelial cells,Mesen,APSE,APSE-control-pancreas-01-a,pancreas,primary,nuclei
APSE-control-pancreas-01-a_CTCCCTCTCGCCCAGA-1,5712,2098,53,F,fibrotic-panc,Endothel_venous,fibrosis,0.004201681,head_PPI,Liebig,Endothelial cells,Mesen,APSE,APSE-control-pancreas-01-a,pancreas,primary,nuclei
APSE-control-pancreas-01-a_CAATACGTCCACTGGG-1,5540,2388,53,F,fibrotic-panc,Endothel_lymphatic,fibrosis,0.006859206,head_PPI,Liebig,Endothelial cells,Mesen,APSE,APSE-control-pancreas-01-a,pancreas,primary,nuclei
APSE-control-pancreas-01-a_AGGGCCTTCTTTGCTA-1,5392,2514,53,F,fibrotic-panc,Endothel_venous,fibrosis,0.007418398,head_PPI,Liebig,Endothelial cells,Mesen,APSE,APSE-control-pancreas-01-a,pancreas,primary,nuclei
APSE-control-pancreas-01-a_TTACTGTTCCAAGCTA-1,5290,2213,53,F,fibrotic-panc,Endothel_lymphatic,fibrosis,0.005671078,head_PPI,Liebig,Endothelial cells,Mesen,APSE,APSE-control-pancreas-01-a,pancreas,primary,nuclei
APSE-control-pancreas-01-a_TGCGATACAGTTGTCA-1,5081,2396,53,F,fibrotic-panc,Endothel_lymphatic,fibrosis,0.003936233,head_PPI,Liebig,Endothelial cells,Mesen,APSE,APSE-control-pancreas-01-a,pancreas,primary,nuclei
APSE-control-pancreas-01-a_CTGGTCTGTCTTGGTA-1,5063,2353,53,F,fibrotic-panc,Endothel_arterial,fibrosis,0.002765159,head_PPI,Liebig,Endothelial cells,Mesen,APSE,APSE-control-pancreas-01-a,pancreas,primary,nuclei
APSE-control-pancreas-01-a_CGAGTTATCGCCACTT-1,5000,2268,53,F,fibrotic-panc,Endothel_lymphatic,fibrosis,0.003800000,head_PPI,Liebig,Endothelial cells,Mesen,APSE,APSE-control-pancreas-01-a,pancreas,primary,nuclei
APSE-control-pancreas-01-a_GGACGTCTCCTTCGAC-1,4919,2206,53,F,fibrotic-panc,Endothel_lymphatic,fibrosis,0.005082334,head_PPI,Liebig,Endothelial cells,Mesen,APSE,APSE-control-pancreas-01-a,pancreas,primary,nuclei


In [48]:
data_espace_seurat <- CreateSeuratObject(counts = aggregated_counts, meta.data = data_espace_metadata)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [49]:
save(data_espace_seurat, file = "data_espace_seurat.RData")